In [ ]:
import sys 
import matplotlib.pyplot as plt

pml = pulsedmeasurementlogic
pmal = pulsedmasterlogic
import pprint

import logic.pulsed.pulse_objects as po
import logic.pulsed.sampling_function_defs.basic_sampling_functions as functions
import time
import os

from tqdm import tqdm

PODMR - repetitions at each point

Necessary classes and functions

In [ ]:
class PulseSequence:
    '''
    A pulse sequence to be loaded that is made of PulseBlock instances. The pulse blocks can be repeated
    as well and multiple can be added.
    '''
    def __init__(self):
        self.pulse_dict = {'d0':[], 'd1':[], 'd2':[], 'd3':[], 'd4':[], 'd5':[], 'd6':[], 'd7':[], 'a0':[], 'a1':[]}

    def append(self, block_list):
        '''
        append a list of tuples of type: 
        [(PulseBlock_instance_1, n_repetitions), (PulseBlock_instance_2, n_repetitions)]
        '''
        for block, n in block_list:
            for i in range(n):
                for key in block.block_dict.keys():
                    self.pulse_dict[key].extend(block.block_dict[key])

    
class PulseBlock:
    '''
    Small repeating pulse blocks that can be appended to a PulseSequence instance
    '''
    def __init__(self):
        self.block_dict = {'d0':[], 'd1':[], 'd2':[], 'd3':[], 'd4':[], 'd5':[], 'd6':[], 'd7':[], 'a0':[], 'a1':[]}
    
    def append(self, init_length, channels, repetition):
        '''
        init_length in s; will be converted by sequence class to ns
        channels are digital channels of PS in swabian language
        '''
        for i in range(repetition):
            for chn in channels.keys():
                self.block_dict[chn].extend([(init_length/1e-9, channels[chn])])

def set_up_odmr(pi_pulse=100e-9):
    """ 
    @param float clock_frequency: if defined, this sets the frequency of the
                                  clock
    @param str clock_channel: if defined, this is the physical channel of
                              the clock

    @return int: error code (0:OK, -1:error)
    """
    channels = {'d0': 0.0 , 'd1': 0.0 , 'd2': 0.0 , 'd3': 0.0 , 'd4': 0.0 , 'd5': 0.0 , 'd6': 0.0 , 'd7': 0.0 , 'a0': 0.0, 'a1': 0.0}
    clear = lambda x: {i:0.0 for i in x.keys()}
    d_ch = lambda x: f'd{x}'
    a_ch = lambda x: f'a{x}'

    seq = PulseSequence()
    block_1 = PulseBlock()

    channels = clear(channels)
    channels[a_ch(pulsestreamer._laser_analog_channel)] = pulsestreamer._laser_power_voltage
    channels[d_ch(pulsestreamer._mw_switch)] = 1.0
    block_1.append(init_length = pi_pulse, channels = channels, repetition = 1)
    
    channels = clear(channels)
    channels[a_ch(pulsestreamer._laser_analog_channel)] = pulsestreamer._laser_power_voltage
    block_1.append(init_length = 1e-6, channels = channels, repetition = 1)
    
    channels = clear(channels)
    channels[d_ch(pulsestreamer._laser_channel)] = 1.0
    channels[a_ch(pulsestreamer._laser_analog_channel)] = pulsestreamer._laser_power_voltage
    channels[d_ch(pulsestreamer._pixel_start)] = 1.0 # pulse to TT channel detect
    block_1.append(init_length = 3e-6, channels = channels, repetition = 1)
    
    channels = clear(channels)
    channels[a_ch(pulsestreamer._laser_analog_channel)] = pulsestreamer._laser_power_voltage
    block_1.append(init_length = 1.5e-6, channels = channels, repetition = 1)

    seq.append([(block_1, 1)])

    pulse_dict = seq.pulse_dict

    pulsestreamer.load_swabian_sequence(pulse_dict)
    return pulsestreamer._seq

def set_up_next_trigger():
    """ 
    @param float clock_frequency: if defined, this sets the frequency of the
                                  clock
    @param str clock_channel: if defined, this is the physical channel of
                              the clock

    @return int: error code (0:OK, -1:error)
    """
    channels = {'d0': 0.0 , 'd1': 0.0 , 'd2': 0.0 , 'd3': 0.0 , 'd4': 0.0 , 'd5': 0.0 , 'd6': 0.0 , 'd7': 0.0 , 'a0': 0.0, 'a1': 0.0}
    clear = lambda x: {i:0.0 for i in x.keys()}
    d_ch = lambda x: f'd{x}'
    a_ch = lambda x: f'a{x}'

    seq = PulseSequence()
    block_1 = PulseBlock()

    channels = clear(channels)
    channels[a_ch(pulsestreamer._laser_analog_channel)] = pulsestreamer._laser_power_voltage
    channels[d_ch(pulsestreamer._pixel_stop)] = 1.0
    block_1.append(init_length = 1e-3, channels = channels, repetition = 1)
    
    channels = clear(channels)
    channels[a_ch(pulsestreamer._laser_analog_channel)] = pulsestreamer._laser_power_voltage
    block_1.append(init_length = 1e-3, channels = channels, repetition = 1)
    
    seq.append([(block_1, 1)])

    pulse_dict = seq.pulse_dict

    pulsestreamer.load_swabian_sequence(pulse_dict)
    return pulsestreamer._seq

Run measurement

In [ ]:
var_list = np.linspace(2.844e9, 2.876e9, 75, endpoint=True)
mw_power = -10
pi_pulse=394e-9
num_runs = 147000

mw_source_smbv.set_list(var_list, mw_power) # 
mw_source_smbv.list_on()
laser_pulses = len(var_list)

bin_width_s = 1e-9
record_length_s = 3e-6

ret_val = time_tagger.configure_recorder(
                mode=11, # pulsed mode
                params={'laser_pulses': laser_pulses,
                        'bin_width_s': bin_width_s,
                        'record_length_s': record_length_s,
                        'max_counts': 1} )


time_tagger.start_recorder(arm=True)

set_up_next_trigger()
pulsestreamer.pulser_on(n=2)

podmr = set_up_odmr(pi_pulse)

for i in tqdm(range(laser_pulses)):
    pulsestreamer.pulser_on(n=num_runs, final=pulsestreamer._mw_trig_final_state)
    d =time.time()
    while True:
        if time_tagger.recorder.getHistogramIndex() > i or (time.time()-d)>10 or time_tagger.recorder.getCounts()>0:
            time.sleep(0.001)
            break

pulsed_meas = time_tagger.get_measurements()[0]
pulsestreamer.pulser_off()         
mw_source_smbv.off()   

Plot Jupyter data

In [ ]:
data,_ =afm_scanner_logic.analyse_pulsed_meas(pulsedmasterlogic.analysis_settings, pulsed_meas)

# fit = afm_scanner_logic._fitlogic.make_lorentzian_fit(var_list,data,estimator=afm_scanner_logic._fitlogic.estimate_lorentzian_dip)
fit = afm_scanner_logic._fitlogic.make_lorentziandouble_fit(var_list,data,estimator=afm_scanner_logic._fitlogic.estimate_lorentziandouble_N15)
print(fit.fit_report())
# lm,_ = fitlogic.make_lorentzian_model()
lm,_ = fitlogic.make_lorentziandouble_model()
plt.plot(var_list, data,'bo')
x = np.linspace(var_list[0],var_list[-1],10000, endpoint=True)
plt.plot(x, lm.eval(fit.params, x=x),'r-')
plt.show()

Plot SPM data

In [ ]:
d = afm_scanner_logic._pulsed_scan_array['pulsed_fw']['data']
x = afm_scanner_logic._pulsed_scan_array['pulsed_fw']['coord2_arr']
data = d[0,0,:]

In [ ]:
fit = afm_scanner_logic._fitlogic.make_lorentziandouble_fit(x,data,estimator=afm_scanner_logic._fitlogic.estimate_lorentziandouble_N15)
print(fit.fit_report())
lm,_ = fitlogic.make_lorentziandouble_model()
plt.plot(x, data,'bo')
x = np.linspace(x[0],x[-1],10000, endpoint=True)
plt.plot(x, lm.eval(fit.params, x=x),'r-')

plt.show()